In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/MyDrive/PROJECTS/vae')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip /content/gdrive/MyDrive/DATASETS/Celeb_A/celeba-dataset.zip

In [3]:
import tensorflow as tf
import os

DATA_DIR = '/content/img_align_celeba/img_align_celeba'
BATCH_SIZE = 256

def preprocessing(x):
    x = x/255
    return (x,tf.constant([]))

dataset_train, dataset_val  = tf.keras.utils.image_dataset_from_directory(
    directory=DATA_DIR,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=None,
    image_size=(128,128),
    shuffle=True,
    seed=42,
    validation_split = 0.05,
    subset = 'both',)

n_trainingssamples = len(dataset_train)

dataset_train = dataset_train.map(preprocessing).batch(BATCH_SIZE)
dataset_val = dataset_val.map(preprocessing).batch(BATCH_SIZE)

Found 202599 files belonging to 1 classes.
Using 192470 files for training.
Using 10129 files for validation.


In [ ]:
from experiments.celebA.architecture import build_model
from experiments.visualization import VisualizeReconstrcutionCallback, GatherGifFramesCallback

RESULT_DIR = '/content/gdrive/MyDrive/PROJECTS/vae/experiments/celebA/results'

vae = build_model()

callbacks = [
    VisualizeReconstrcutionCallback(images=dataset_val.as_numpy_iterator().next()[0][:9], freq=1,
                                    save_dir=os.path.join(RESULT_DIR, 'train_reconstruction')),
    tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss'),
    tf.keras.callbacks.ModelCheckpoint(os.path.join(RESULT_DIR, 'checkpoints', '{epoch:04d}.tf'),
                                        vebose=0,
                                        save_freq=int(n_trainingssamples/BATCH_SIZE))
]

vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
vae.fit(x=dataset_train, batch_size=BATCH_SIZE, epochs=100, callbacks=callbacks, validation_data=dataset_val)

Output hidden; open in https://colab.research.google.com to view.